## Data preparation

In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

2023-06-15 07:56:58.832782: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

In [3]:

train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['unsupBow.feat',
 'urls_unsup.txt',
 'unsup',
 'urls_neg.txt',
 'neg',
 'labeledBow.feat',
 'pos',
 'urls_pos.txt']

In [4]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [5]:
batch_size = 1024
seed = 12345
train_ds = tf.keras.utils.text_dataset_from_directory(
                            'aclImdb/train', batch_size=batch_size, 
                            validation_split=0.2,
                            subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
                            'aclImdb/train', batch_size=batch_size, 
                            validation_split=0.2,
                            subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [6]:
for text_batch, label_batch in train_ds:
    print(label_batch[0].numpy())
    print(text_batch.numpy()[0])
    break

1
b"The first von Trier movie i've ever seen was breaking the waves. Sure a nice movie but it definitely stands in the shadow of europa. Europa tells a story of a young German-American who wants to experience Germany just after the second world war. He takes a job that his uncle has arranged for him as a purser on a luxues train. Because of his job, he travels all through an almost totally destroyed germany, meeting with the killing of traitors, and hunt for former nazi party members. The society is suffering from corruption. His uncle has narrowed his conciousness by focussing on the job he has also as a purser on the train. By coincidence the main character get involved in bombing and terrorism by a group called 'werewolves' they put pressure on him to help them placing bombs on trains. The atmosphere is astounding. The viewer is taken from scene to scene by a man attempting to put the viewer under hypnosis and then counting to wake you up in a new scene. Just when you think you've s

In [7]:
for text_batch, label_batch in train_ds:
    print(label_batch[0].numpy())
    print(text_batch.numpy()[0].decode('ascii'))
    break

0
I read the comment of Chris_m_grant from United States.<br /><br />He wrote : " A Fantastic documentary of 1924. This early 20th century geography of today's Iraq was powerful."<br /><br />I would like to thank Chris and people who are interested in Bakhtiari Nomads of Iran, the Zagros mountains and landscapes and have watched the movie Grass, A Nation's battle for life. These traditions you saw in the movie have endured for centuries and will go on as long as life endures. I am from this region of Iran myself. I am a Bakhtiari. <br /><br />Chris, I am sorry to bother you but Bakhtiari region of Zardkuh is in Iran not in Irak as you mentioned in your comment. Iran and Irak are two different and distinct countries. Taking an Iranian for an Irankian is almost like taking an American for an Mexican. Thanks,<br /><br />Ziba


In [8]:
vocab_size   = 20000
sequence_len = 200

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

vectorization = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_len,
)

vectorization.adapt(train_ds.map(lambda text, label: text))

In [9]:
output = custom_standardization(text_batch.numpy()[0].decode('ascii'))
print(output.numpy().decode())

i read the comment of chrismgrant from united states  he wrote   a fantastic documentary of 1924 this early 20th century geography of todays iraq was powerful  i would like to thank chris and people who are interested in bakhtiari nomads of iran the zagros mountains and landscapes and have watched the movie grass a nations battle for life these traditions you saw in the movie have endured for centuries and will go on as long as life endures i am from this region of iran myself i am a bakhtiari   chris i am sorry to bother you but bakhtiari region of zardkuh is in iran not in irak as you mentioned in your comment iran and irak are two different and distinct countries taking an iranian for an irankian is almost like taking an american for an mexican thanks  ziba


In [10]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorization(text), label

train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)

In [11]:
for text_batch, label_batch in train_ds:
    print(label_batch[0].numpy())
    print(text_batch.numpy()[0])
    break

1
[   44    22    38   330     1   209    66   115    34   182    30    65
   971     3  1279  5960    29    55   413   284   498    91     5    30
   208     1     3     1     2  2978     5     2  3438   818  5742     2
 10008     7    21  8070    32     1     1     4   989   299  3034     3
    40    81     9    13    90     8 10586    89  2152    22    68    66
     2  5511  2388     2  1511 17410    56    11    13     2    84    17
   115    22   400  4328     9    13    90    31     1  2525     1     3
  5099    31     2    58    64     1  2643   421     9     3   172     1
   928   633    13 12271   209   339   360    10   585    86    56    10
  5366  1829 12960    13     4 14897    15     4   172    36    13   457
    41   299   837   779    18    31     2    58   152   178     6    27
  3321    16   927    61    13  3376     4   958   521   141    92    11
    13   337     4    52    52 12062   354    41     4    52     1  1265
    20     2   171     5     2  1493    36     1 

In [12]:
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)

## Training

In [13]:

from tensorflow.keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate) 
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [14]:
 # Two seperate embedding layers, one for tokens, one for token index (positions)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [15]:
from tensorflow import keras

embed_dim = 128  # Embedding size for each token
num_heads = 6    # Number of attention heads
ff_dim = 128     # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(sequence_len,))
embedding_layer = TokenAndPositionEmbedding(sequence_len, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 token_and_position_embeddin  (None, 200, 128)         2585600   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 200, 128)         429184    
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_2 (Dropout)         (None, 128)               0     

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(train_ds, batch_size=32, epochs=3, validation_data=val_ds)

Epoch 1/3
20/20 [==============================] - 25s 641ms/step - loss: 0.7280 - accuracy: 0.5276 - val_loss: 0.6552 - val_accuracy: 0.6600
Epoch 2/3
20/20 [==============================] - 11s 564ms/step - loss: 0.5462 - accuracy: 0.7181 - val_loss: 0.3922 - val_accuracy: 0.8270
Epoch 3/3
20/20 [==============================] - 11s 549ms/step - loss: 0.3184 - accuracy: 0.8653 - val_loss: 0.3348 - val_accuracy: 0.8606
